Setup: The purpose of this code is to clean GPS device point data in recreation and tourism settings. In order to successfully run the code you will need:

(i) GPS device data
(ii) a shapefile of your study location

If necessary, you may also need:

(i) a shapefile of a buffer area for GPS device dropbox areas
(ii) a shapefile of a buffer area for GPS device data download areas

The setup assumes that all GPS tracks and files are saved and stored in the same folder with the appropriate projection system.

In [ ]:
import os
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# Get and print the version of arcpy (version used 3.1.2)
print(f"arcpy version: {arcpy.GetInstallInfo()['Version']}")

# Get and print the version of pandas (version used 1.4.4)
print(f"pandas version: {pd.__version__}")

In [ ]:
import arcgis
print(f"arcgis version: {arcgis.__version__}")

In [ ]:
###Set up key paths

##Base directory folder to retrieve files from and save completed files to for the project
base_dir = r"C:\Users\colby\OneDrive\Documents\Research\Grand Canyon\Raw_GPS_23"

##Folder name for the folder containing the GPS data (must be within base directory)
folder_with_data = "GRCA_GPS_TRACKS_2023"

##The geodatabase you wish to use to temporarily store and work with files as you clean the data
target_gdb = r"C:\Users\colby\OneDrive - The Pennsylvania State University\Documents\ArcGIS\Projects\TTRA_Presentation\TTRA_Presentation.gdb"

In [ ]:
##Identify location of relevant shapefiles

##Shapefile of the destination boundary for your study site. Procedure for doing this involved creating a polygon around National Park Service-provided roads and trails in the South Rim and removing entrances/Forest service roads outside the park caught by polygon creation by hand.
destination_boundary = os.path.join(base_dir, "GRCA_SRim_Boundary")

#For some studies, drop boxes may be located within the study boundary. If necessary, create an appropriately sized polygon for removing those points so that they do not generate noise, either by hand or using the buffer tool from the point. In this case, we used the buffer tool set at 150m.
dropbox_buffer = os.path.join(base_dir, "Dropbox_Buffer")

#For some studies, data in GPS devices may be uploaded within the study site and devices may record "ghost" points when they are uploaded at researcher facilities. If necessary, create an appropriately sized polygon for removing points from those locations so that they do not generate noise, either by hand or using the buffer tool from the point. In this case, we used the buffer tool set at 75m from the worker cabins we downloaded points from.
download_buffer = os.path.join(base_dir, "Cabin_Buffer")

In [ ]:
##Set your arcpy environment and permit overwrite

arcpy.env.workspace = target_gdb
arcpy.env.overwriteOutput = True

Section 1: In this section you will do the high level importing and cleaning of your data. This will include importing all of your data into your geodatabase, pre-processing your fields, and removing points based on spatial parameters (i.e., destination boundary, dropbox boundary, download area boundary) relevant to your study site, as applicable.

In [ ]:
###Import data. In order for this to work, all data must be recorded as a shape file.
in_data = os.path.join(base_dir, folder_with_data)
filter = "*.shp"

arcpy.intelligence.BatchImportData(in_data, target_gdb, filter)

In [ ]:
### Combine all separate shapefiles for each unique device/participant into one file

##Set the workspace
workspace = target_gdb

#Making an empty list to put filepaths to feature classes in
feature_classes = []

#Walking workspace recursively checking type, and appending filepath to list
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace,datatype="FeatureClass", type="Point"):
    for filename in filenames:
        desc = arcpy.Describe(os.path.join(dirpath, filename))
        if desc.shapeType == "Point":
            feature_classes.append(os.path.join(dirpath, filename))

#Filename for saving
output = "raw_points"

#merging my list of feature classes to a new dataset
arcpy.Merge_management(feature_classes, output)

In [ ]:
##It is unnecessary to retain most fields recorded by GPS devices, however, specific needs for each project may vary. Below are the minimum necessary fields to run the code.

process_raw = os.path.join(target_gdb, "raw_points")

raw_points = pd.DataFrame.spatial.from_featureclass(process_raw)

##Select the field for the object identification (OID) created by ArcGIS
OID = "OBJECTID"

##Select the field for group identification (GID), which refers the participant's unique ID
GID = "tident"

##Select the field(s) for time
time= "ltime"

##Select the field from
shape = "SHAPE"

##Retained fields
retained_fields = [OID,GID,time,shape]

##Create dataframe of necessary points only
raw_points = raw_points[retained_fields]

##Export datapoints back to geodatabase
raw_points.spatial.to_featureclass(os.path.join(target_gdb, "processed_points"))

##Print pandas dataframe
raw_points

In [ ]:
# Use Convert Time Field to convert “ltime” to Date format (labeling may vary). Input time format may vary by device.

in_table = "processed_points"
input_time_field = time
input_time_format = "yyyy/MM/dd HH:mm:ss"
output_time_field = "ltime_converted"

arcpy.management.ConvertTimeField(in_table, input_time_field, input_time_format, output_time_field)

In [ ]:
# Clip new merged shapefile using a polygon of the study area (with the survey locations cut out of the polygon – use continue feature tool)

in_features = "processed_points"
clip_features = destination_boundary
out_feature_class = "processed_points_clip"

arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

In [ ]:
## Remove points around dropbox using 150m buffer

in_features = "processed_points_clip"
erase_features = dropbox_buffer
out_feature_class = "processed_points_clip_dropbox"

arcpy.analysis.PairwiseErase(in_features, erase_features, out_feature_class)

In [ ]:
###Remove points around cabin using 75m buffer

in_features = "processed_points_clip_dropbox"
erase_features = download_buffer
out_feature_class = "processed_points_clip_dropbox_cabin"

arcpy.analysis.PairwiseErase(in_features, erase_features, out_feature_class)

Section 2: The preceding section provides the highest level of data cleaning, however, the subsequent sections allow the researcher to dig deeper into errors that may occur from working with GPS devices. Section 2 relies on automatic commands to systematically remove points from likely errors (e.g., devices being turned off, noise). It will require defining assumptions that are pertinent to the set up of your GPS devices and reasonable behaviors of your participants.

In [ ]:
# Use the Points to Track Segments tool to create a new set of features where all points are converted to track segments. Use the participant identification number (in this case, “tident”) as the group field. Deselect Error On Duplicate Timestamps.

in_features = "processed_points_clip_dropbox_cabin"
date_field = "ltime_converted"
out_feature_class = "processed_lines"
group_field = GID
include_velocity = "INCLUDE_VELOCITY"
error_on_duplicate_timestamps = "ALLOW_DUPLICATE_TIMESTAMPS"

arcpy.intelligence.PointsToTrackSegments(in_features, date_field, out_feature_class, group_field, error_on_duplicate_timestamps = "ALLOW_DUPLICATE_TIMESTAMPS")

In [ ]:
##Assumptions:

##The GPS devices used in this study relied on 15 second intervals and the fastest method for travel were by car', therefore, we assumed that distances between points above 465 meters (i.e., 70 mph) were likely due to error. Change the field preceding the ">" sign to match the field indicating distance between points.

speed_assumption = "distance_m > 465"

##The GPS devices used in this study relied on 15 second intervals, therefore, we assumed that recordings with intervals above 20 seconds were indicators of error. Change the field preceding the ">" sign to match the field indicating time between points.

time_assumption = "dt_sec > 20"

In [ ]:
# Select all attributes (line segments) with a distance greater than 465 meters (traveling greater than 70 miles per hour). Delete the selection.

in_layer_or_view = "processed_lines"
selection_type = "NEW_SELECTION"
where_clause = speed_assumption

Distance_Selection = arcpy.management.SelectLayerByAttribute(in_layer_or_view, selection_type, where_clause)


##Delete selected attributes.
arcpy.management.DeleteFeatures(Distance_Selection)

In [ ]:
# Create new text fields equal to your group identification plus the unique point identification for each entry for further examination. First make sure your fields, such as identification, are text. These expressions should occur by default in your data by now, but revise them (i.e., the group ID, object ID) if they do not.

in_table = "processed_lines"
field = "Text_OID"
expression = "!OBJECTID!"

arcpy.management.CalculateField(in_table, field, expression, field_type = "TEXT")

##Now make a field that combines Group Identification+OBJECTID.

in_table = "processed_lines"
field = "ID_2"
expression = "!group_id! + !Text_OID!"

arcpy.management.CalculateField(in_table, field, expression)

###Remove Unnecessary Fields (again)

in_table = "processed_lines"
fields = ["Text_OID"]

arcpy.DeleteField_management(in_table, fields)

In [ ]:
# Use the Feature Vertices to Points tool to transform the line segments of participant feature classes to points. Again, stay working in the same geodatabase. Use start vertex as the Point Type. Add the new point-based feature classes to the map.

in_features = "processed_lines"
out_feature_class = "processed_points"
point_location = "START"

arcpy.management.FeatureVerticesToPoints(in_features, out_feature_class, point_location)

In [ ]:
# Use select by attributes to delete all rows (points) containing duration values greater than 20. This won't remove all errors, but it is a good start.

in_layer_or_view = "processed_points"
selection_type = "NEW_SELECTION"
where_clause = time_assumption

Time_Selection = arcpy.management.SelectLayerByAttribute(in_layer_or_view, selection_type, where_clause)

arcpy.management.DeleteFeatures(Time_Selection)

Section 3: The preceding section is helpful for automatic removal of GPS tracks within ArcGIS Pro. However, GPS data can be noisy and may require some manual effort to identify lingering errors. For example, in the Grand Canyon study, some people left the study site for some time (e.g., to shop, camp outside the park boundaries) and the returned. Their data is still valuable, however, it could be mixed up with other data that is from device error (e.g., devices that recorded points in illogical destinations due to noise or topography), user error (e.g., a participant who appeared to have a device on them as they did a helicopter tour of the park) or researcher error (e.g., points recorded on GPS devices retrieved from visitor centers). The code below is intended to facilitate manual examination of data points that express unique or illogical behavior to help determine if they are valuable or warrant removal because they are derived from some form of error.

In [ ]:
##Parse .csv for analysis and create data labels for clarity indicating the labels for points, participant identification, and field for time

processed_points_2 = os.path.join(target_gdb, "processed_points")

df = pd.DataFrame.spatial.from_featureclass(processed_points_2)
df.rename(columns={'OBJECTID':'FID','group_id': 'GID', 'd_start':'dtime'},inplace=True)

##Convert time to proper time field format
df['dtime'] = pd.to_datetime(df['dtime'])

##Sort data by field and ID
df.sort_values(by=['GID', 'dtime'], inplace=True)
df

##Dataframe copy

seq_timedf = df.copy()
seq_timedf

In [ ]:
##Use the shift command to record the subsequently recorded point for each point from each participant

seq_timedf.sort_values(by=['GID', 'dtime'], inplace=True)
seq_timedf[['GID_2', 'dtime_2']] = seq_timedf.groupby('GID')[['GID', 'dtime']].shift(-1)
seq_timedf['GID_2'] = seq_timedf['GID_2'].astype(str).str.split('.0')
seq_timedf['GID_2'] = seq_timedf['GID_2'].str.get(0)
seq_timedf

In [ ]:
##Identify a length of time between points that is likely to have occurred because of error, manually readjusting based on the subsequent steps until you consistently stop identifying errors. In our case, this was 16 hours.

seq_timedf2 = seq_timedf.copy()
seq_timedf2['POI_timedelta'] = seq_timedf2['dtime_2'] - seq_timedf2['dtime']
seq_time_filt = seq_timedf2['POI_timedelta'] > '0 days 16:00:00'

##Createa field to serve as a filter to identify when this occurs for each recorded point
seq_timedf2['POI_timedelta_filt'] = seq_time_filt
seq_timedf2['POI_timedelta_filt'] = seq_timedf2['POI_timedelta_filt'].astype(int)
seq_timedf2['POI_timedelta_filt'] = seq_timedf2.groupby('GID')['POI_timedelta_filt'].cumsum()
seq_timedf2['GID_2'] = seq_timedf2['GID'].astype(str)+'.'+seq_timedf2['POI_timedelta_filt'].astype(str)
seq_timedf2

In [ ]:
##Create dataframe containing filter for time delta for manual evaluation

seq_time_filt2 = seq_time_filt==True
df10 = seq_timedf2.copy()
df10 = pd.DataFrame(df10.loc[seq_time_filt2])
df10

Determine errant points and tracks based off of time and place that need to be deleted. Make your determination based on your best judgement due to (i) length of time from preceding points and (ii) visual examination of the points in ArcGIS. You can do this by looking the point field for seemingly errant points in ArcGIs and confirming if they indicate a spatial pattern that may result from error. For example, in our case, if points occurred near cabin or GPS device drop-off location/far away from preceding points in time and space.

To conduct the manual examination, follow the code blocks below for each participant ID that was identified above as being susceptible to error. Record the entire tracks and individual points that caused issues below. Depending on your comfort with Python, you can follow the code below to delete these tracks, or, if you prefer, you can delete them by hand in ArcGIS.

In [ ]:
##Identify tracks with errors. This deeper dive may sometimes help you identify errors from other steps in your process. For example, technicians may save errant tracks or mislabel them. These tracks may entirely be the results of error entirely and warrant removal, whereas other times they simply were mislabeled. Below are mislabeled tracks for deletion with justification based on visual examination:

##Current Track: 22 Jun is 239812 and researcher error and should be deleted
##Current Track: 05 Jun is 111212 and researcher error and should be deleted

##Below are acceptable mislabeled tracks:
##Current Track: 28 May 2 is 059712
##Current Track: May 30 2 is 079812
##Current Track: 10 Jun 2 is 152212
##Current Track: 16 Jun 2 is 192212

In [ ]:
##Identify points with errors.

##Points with issues final point at Main entrance
GID_last_points = [172012, 172112, 175812, 172412]

##Points with issues at final two points at Main entrance
GID_two_last_points =[179812]

##Points with issues at final three points at Main Entrance
GID_three_last_points = [171712, 172312]

##Points with issues at final four points at Main Entrance
GID_four_last_points = [175112]


##Questionable points:
##179912 has a series of points three days after the rest that are drive from GPD dropbox up desert view (don't end at cabins)
##059512 has a series of points from main entrance that lead to the cabin about a day from all other points
##055522 has a series of points from main entrance that lead to the cabin about a day from all other points
##231712 much of the device error around GPS dropbox

##Odd behaviors, but data seems worth retaining:
##010812 has lapses and stays in a cabin for a long time but doesn't seem like researcher error
##016712 appears to have camped outside the park and returned
##176312 appears to have camped outside the park and returned
##015012 appears to have camped outside the park and returned
##170912 appears to have camped outside the park and returned
##131712 appears to have camped outside the park and returned
##091112 appears to have camped outside the park and returned
##015512 odd lag/error with normal behavior before and after (perhaps device turned off while in campground)
##177012 appears to have camped outside the park and returned
odd_data = ["010812", "016712", "176312", "015012", "170912", "131712", "091112", "015512","177012"]

##Note on when cleaning stopped: Inspection ended at timedelta of 16:35 because . Therefore, all GIDs with time delta greater than that were inspected in this dataset.

In [ ]:
###Method for searching for errors in GID. Use the code below to create a dataframe listing all points from participant identifications flagged with having potential error individually. Open ArcGIs and use the "Search by Attribute Function" to highlight all points from a participant identification. Visually examine all points to identify a pattern that may explain the apparent error. Determine if the pattern suggests behavior worth retaining (e.g., visitor left the destination but later returned) and record it above for record. If the pattern appears to reveal an error instead (e.g., one or two points appear at the end of the track in a distant location, the device appears to have been dropped off at the dropbox and recorded the researcher driving through the destination), then record it appropriately above.

error_GID = "177012"
seq_timedf10 = df.copy()
seq_timedf_10 = seq_timedf10['GID'] == error_GID
seq_timedf_10 = pd.DataFrame(seq_timedf10.loc[seq_timedf_10])
seq_timedf_10

In [ ]:
##Create dataframe of all values that come after large time delta with odd data excluded (i.e., data that was not from error, but odd user behavior), which will then inherently only retain data with errors.

df11 = seq_timedf2.copy()
seq_time_filt3 = df11["POI_timedelta_filt"] == 1
df11 = df11.loc[seq_time_filt3]
df11 = df11.groupby('GID',group_keys=False).apply(lambda x:x[1:])
df11 = df11[~df11["GID"].isin(odd_data)]
df11["GID"].value_counts()

In [ ]:
##Create a dataframe of points that should be deleted.

Delete_Points = pd.DataFrame(df11[['FID','SHAPE']])
Delete_Points

In [ ]:
##Create a dataframe of GIDs that need to be deleted (i.e., full participant tracks)

Bad_GID = df.copy()
Bad_GIDs = ["Current Track: 22 JUN 2","Current Track: 05 JUN 2"]
Bad_GID2 = pd.DataFrame(Bad_GID[Bad_GID["GID"].isin(Bad_GIDs)])
Bad_GID2

In [ ]:
##Simplify the dataframe

Bad_GID3 = pd.DataFrame(Bad_GID2[['FID','SHAPE']])
Bad_GID3

In [ ]:
##Concatentate dataframes with points/tracks that must be deleted from points layer

Delete_Points2 = pd.concat([Delete_Points, Bad_GID3])

In [ ]:
##Import of dataframe to ArcGIS geodatabase

Delete_Points2.spatial.to_featureclass(location=os.path.join(target_gdb, "delete_points"))

In [ ]:
##Select the overlapping points between the original shapefile and the shapefile with the points to delete

in_layer = "processed_points"
overlap_type = "INTERSECT"
select_features = "delete_points"

Error_Selection = arcpy.management.SelectLayerByLocation(in_layer, overlap_type, select_features,selection_type = "NEW_SELECTION")

##Delete the points

arcpy.management.DeleteFeatures(Error_Selection)

In [ ]:
##With the data downloaded, do some final house cleaning by examining descriptive attributes

processed_points_3 = pd.DataFrame.spatial.from_featureclass(os.path.join(target_gdb, "processed_points"))
processed_points_3.head()

In [ ]:
##Update fields as needed to proper labeling scheme

processed_points_3.loc[processed_points_3["group_id"] == "Current Track: 28 MAY 2", "group_id"] = "059712"
processed_points_3.loc[processed_points_3["group_id"] == "Current Track: 10 JUN 2", "group_id"] = "152212"
processed_points_3.loc[processed_points_3["group_id"] == "Current Track: 30 MAY 2", "group_id"] = "079812"
processed_points_3.loc[processed_points_3["group_id"] == "Current Track: 16 JUN 2", "group_id"] = "192212"

In [ ]:
##Check trip length for consistency (both minimumum and maximum.

trip_length = processed_points_3.groupby('group_id')['d_start'].agg(['min', 'max'])

# Compute the difference in days between max and min 'd_start'
trip_length['trip_length'] = (trip_length['max'] - trip_length['min'])

trip_length.sort_values(by='trip_length')

In [ ]:
# Delete illogically long or short trips after examining them in ArcGIS Pro
processed_points_3 = processed_points_3[~processed_points_3['group_id'].isin(['155012', '095612'])]

In [ ]:
##Check sample size and number of total observations

uniqGID = processed_points_3['group_id'].nunique()
print("unique GIDs:" + str(uniqGID))
waypoints = processed_points_3.shape[0]
print("# of waypoints:" + str(waypoints))

In [ ]:
processed_points_3.spatial.to_featureclass(location=os.path.join(target_gdb, "cleaned_points"))

In [ ]:
##Delete obvious erroneous points manually (delete key in ArcGIS):
# 61 points around GPS device dropbox location outside the buffer range were manually deleted

# COMMENT:Below are possible errors missed after performing the above steps (There were fewer than 100 errors that I identified, which is great for dataset of 350,000+):
#   1. 11 points appear in a cluster in the canyon far off trail with no apparent lead up on Bright Angel Trailhead near Powell Point. These may be deleted, but again, that may be better to leave to the individual researcher based on their needs/analysis.
#   2. 6 Points next to Navajo overlook that seem excessively far into the canyon even accounting for error by the same device. These may be deleted, but again, that may be better to leave to the individual researcher based on their needs/analysis.